### Bayesian Neural Network for Kp Prediction


In [ ]:
import tensorflow as tf
import tensorflow_probability as tfp
import pandas as pd
import numpy as np


In [ ]:
data = pd.read_csv("data/dsc_fc_summed_spectra_2023_v01 2.csv", \
delimiter = ',', parse_dates=[0], \
infer_datetime_format=True, na_values='0', \
header = None)

data[0] = pd.to_datetime(data[0], format='%Y-%m-%d %H:%M:%S')

# Set date column as index
data.set_index(0, inplace=True)

# Resample and calculate mean for each hour
hourly_data = data.resample('H').mean()

kp_data = pd.read_csv("KP Data.csv")

# Assuming that 'day' column represents the day of the year (1-365/366), convert it to a proper datetime object
kp_data['date'] = kp_data.apply(lambda row: pd.Timestamp(year=int(row['Year']), month=1, day=1) + pd.Timedelta(days=int(row['Day'])-1, hours=int(row['Hour'])), axis=1)

# Set the new 'date' column as the index
kp_data.set_index('date', inplace=True)

hourly_data_2023 = hourly_data.loc['2023']
kp_data_2023 = kp_data.loc['2023']

data = merged_data = pd.merge(hourly_data_2023, kp_data_2023['Kp'], left_index=True, right_index=True, how='left')
merged_data_imputed = merged_data.fillna(merged_data.median()) #Impute missing values with median 

df = merged_data_imputed

In [ ]:
# Extracting features and target
X = df.iloc[:, 3:53].values  # Columns 4-53 as features
y = df['Kp'].values    # Kp index as target

# Split the data into train and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train)

#Standardize the data (very important for neural networks)
from sklearn.preprocessing import StandardScaler
scaler_X = StandardScaler().fit(X_train)
X_train = scaler_X.transform(X_train)
X_test = scaler_X.transform(X_test)



### Design Model

In [ ]:
# Define the model
def build_bayesian_model(input_shape):
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(input_shape,)),
        tfp.layers.DenseVariational(100, activation='relu', make_prior_fn=prior, make_posterior_fn=posterior),
        tfp.layers.DenseVariational(20, activation='relu', make_prior_fn=prior, make_posterior_fn=posterior),
        tfp.layers.DenseVariational(10, activation='relu', make_prior_fn=prior, make_posterior_fn=posterior),
        tfp.layers.DenseVariational(5, activation='relu', make_prior_fn=prior, make_posterior_fn=posterior),
        tfp.layers.DenseVariational(1, make_prior_fn=prior, make_posterior_fn=posterior)
    ])
    return model

# Define prior
def prior(kernel_size, bias_size=0, dtype=None):
    n = kernel_size + bias_size
    return lambda t: tfp.distributions.Independent(
        tfp.distributions.Normal(loc=tf.zeros(n, dtype=dtype), scale=1),
        reinterpreted_batch_ndims=1)

# Define posterior
def posterior(kernel_size, bias_size=0, dtype=None):
    n = kernel_size + bias_size
    return tf.keras.Sequential([
        tfp.layers.VariableLayer(2 * n, dtype=dtype),
        tfp.layers.DistributionLambda(lambda t: tfp.distributions.Independent(
            tfp.distributions.Normal(loc=t[..., :n],
                                     scale=tf.nn.softplus(t[..., n:])),
            reinterpreted_batch_ndims=1)),
    ])

# Set units variable, which is required in prior and posterior functions
units = 2000

# Build the model
model = build_bayesian_model(X_train.shape[1])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.000001), loss='mean_squared_error')


In [ ]:
history = model.fit(X_train, y_train, epochs=100, verbose=1, validation_split=0.1)

In [ ]:
# Get samples from the posterior and predict
n_samples = 100
predicted = [model(X_test) for _ in range(n_samples)]

# Compute mean and standard deviation of predictions
predicted_mean = np.mean(predicted, axis=0)
predicted_std = np.std(predicted, axis=0)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming `predicted_mean` is a numpy array or list containing the predicted Kp values
# and y_test is your test set actual Kp values.
predicted_mean = np.squeeze(predicted_mean)  # Ensure predictions are 1D

plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_test, y=predicted_mean)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red')  # Diagonal line
plt.xlabel('Actual Kp')
plt.ylabel('Predicted Kp')
plt.title('Actual vs. Predicted Kp Values')
plt.show()